<a href="https://colab.research.google.com/github/cwiet/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [10]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard 

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
      )
  
  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v4(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),  
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),  
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),  
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax'),
  ])

In [8]:
model = get_cnn_v4(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 5s 17ms/step - loss: 2.9243 - accuracy: 0.2191 - val_loss: 1.3602 - val_accuracy: 0.5874
Epoch 2/5
272/272 [==============================] - 4s 16ms/step - loss: 1.0327 - accuracy: 0.6759 - val_loss: 0.2854 - val_accuracy: 0.9171
Epoch 3/5
272/272 [==============================] - 4s 15ms/step - loss: 0.4724 - accuracy: 0.8510 - val_loss: 0.1429 - val_accuracy: 0.9538
Epoch 4/5
272/272 [==============================] - 4s 15ms/step - loss: 0.3239 - accuracy: 0.9004 - val_loss: 0.0929 - val_accuracy: 0.9715
Epoch 5/5
272/272 [==============================] - 4s 15ms/step - loss: 0.2384 - accuracy: 0.9279 - val_loss: 0.0684 - val_accuracy: 0.9779


0.9609977324263038

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),  
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),  
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),  
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax'),
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
      )
  
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [19]:
space = {
  'batch_size': hp.quniform('batch_size', 100, 200, 10), 
  'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
  'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
  'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

  'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
  'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    25,
    Trials()
)

{'batch_size': 190.0, 'dropout_cnn_block_one': 0.4950195079977633, 'dropout_cnn_block_three': 0.3900399364749925, 'dropout_cnn_block_two': 0.3721592966750244, 'dropout_dense_block_one': 0.30333982743293936, 'dropout_dense_block_two': 0.6729698790741743}
accuracy=0.8725623488426208
{'batch_size': 100.0, 'dropout_cnn_block_one': 0.47877518617808723, 'dropout_cnn_block_three': 0.3924090040839528, 'dropout_cnn_block_two': 0.36958069770558044, 'dropout_dense_block_one': 0.522237545038053, 'dropout_dense_block_two': 0.4139369956308325}
accuracy=0.9149659872055054
{'batch_size': 200.0, 'dropout_cnn_block_one': 0.31981264690482575, 'dropout_cnn_block_three': 0.3648683842373779, 'dropout_cnn_block_two': 0.41042995051894626, 'dropout_dense_block_one': 0.5503203117415837, 'dropout_dense_block_two': 0.5775925959875926}
accuracy=0.9643990993499756
{'batch_size': 150.0, 'dropout_cnn_block_one': 0.4350812125751746, 'dropout_cnn_block_three': 0.4869732141101889, 'dropout_cnn_block_two': 0.476589334927